# Split Folders

In [1]:
import os
import splitfolders 

input_dir = os.path.join('D:/MedicalMNIST')
output_dir = os.path.join('D:/MedicalMNIST_splitted')

splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.8, .2), group_prefix=None) 

Copying files: 588 files [00:04, 170.00 files/s]

KeyboardInterrupt: 

# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os
import cv2
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Input, Flatten, Dense, Reshape

# Load the Data

In [3]:
train_dir = os.path.join('D:/MedicalMNIST_splitted/train')
test_dir = os.path.join('D:/CT Img/CT_split/val')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(44, 44),
                                                   batch_size= 47163,
                                                   class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,target_size=(44, 44),
                                                   batch_size=11791,
                                                   class_mode='categorical')

In [ ]:
x_train, y_train = next(train_generator)
x_test, y_test  = next(test_generator)
labels = [np.where(i==1)[0][0] for i in example_labels]

In [ ]:
n_examples = 500
example_images = x_test[0:n_examples]
example_labels = y_test[0:n_examples]

f, axs = plt.subplots(1, 5, figsize=(15, 4))
for j in range(len(axs)):
    axs[j].imshow(example_images[j+42], cmap='binary')
    axs[j].axis('off')

# Dimensionality Reduction

In [ ]:
encoded_dim = 2

encoder = Sequential([
    Conv2D(input_shape=(44,44,3), filters= 64, kernel_size = (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D( (2, 2), padding='same'),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    Flatten(),
    Dense(encoded_dim)
])

encoder.summary()

In [ ]:
pretrain_encodings = encoder(example_images).numpy()

In [ ]:
class_names = np.array(['AbdomenCT', 'BreastMRI', 'ChestCT','CXR','Hand','HeadCT'])
f, ax = plt.subplots(1, 1, figsize=(15, 8))
sns.scatterplot(pretrain_encodings[:, 0],
                pretrain_encodings[:, 1],
                hue=class_names[labels], ax=ax,
                palette=sns.color_palette("colorblind", 6));
ax.set_xlabel('Encoding dimension 1'); ax.set_ylabel('Encoding dimension 2')
ax.set_title('Encodings of example images before training');

In [ ]:
decoder = Sequential ([
    Dense(1936, activation = 'relu', input_shape=(encoded_dim,)),
    Reshape((11,11,16)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(3, (3, 3), padding='same') 
])

decoder.summary()

In [ ]:
autoencoder_dr = Model(inputs = encoder.inputs, outputs = decoder(encoder.outputs))

autoencoder_dr.compile(loss = 'mse')

autoencoder_dr.fit(x_train, x_train, epochs = 10, verbose=1)

In [ ]:
posttrain_encodings = encoder(example_images).numpy()

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))
sns.scatterplot(pretrain_encodings[:, 0],
                pretrain_encodings[:, 1],
                hue=class_names[labels], ax=axs[0],
                palette=sns.color_palette("colorblind", 6));
sns.scatterplot(posttrain_encodings[:, 0],
                posttrain_encodings[:, 1],
                hue=class_names[labels], ax=axs[1],
                palette=sns.color_palette("colorblind", 6));

axs[0].set_title('Encodings of example images before training');
axs[1].set_title('Encodings of example images after training');

for ax in axs: 
    ax.set_xlabel('Encoding dimension 1')
    ax.set_ylabel('Encoding dimension 2')
    ax.legend(loc='upper right')

In [ ]:
reconstructed_img = autoencoder_dr(example_images)

In [ ]:
f, axs = plt.subplots(2, 5, figsize=(15, 8))
for j in range(5):
    axs[0, j].imshow(example_images[j+20], cmap='binary')
    axs[1, j].imshow(reconstructed_img[j+20].numpy().squeeze(), cmap='binary')
    axs[0, j].axis('off')
    axs[1, j].axis('off')

# Image Denoising

In [ ]:
noise_factor = 0.2
x_train_rand = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_rand = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

In [ ]:
f, axs = plt.subplots(1, 5, figsize=(15, 8))
for j in range(len(axs)):
    axs[j].imshow(x_train_rand[j+5], cmap='binary')
    axs[j].axis('off')

In [ ]:
encoder_noise = Sequential([
    
    Conv2D(input_shape = (44,44,3), filters = 256, kernel_size = (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
])

encoder_noise.summary()

In [ ]:
decoder_noise = Sequential ([
    Conv2D(input_shape=(11, 11, 64), filters = 128, kernel_size=(3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(3, (3, 3), activation='sigmoid', padding='same')
])

decoder_noise.summary()

In [ ]:
autoencoder_noise = Model(inputs = encoder_noise.inputs, outputs = decoder_noise(encoder_noise.outputs))
autoencoder_noise.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder_noise.fit(x_train_rand, x_train,
                epochs=10,
                verbose=1)

In [ ]:
x_predict = autoencoder_noise.predict(x_test_rand)

In [ ]:
f, axs = plt.subplots(2, 5, figsize=(15, 8))
for j in range(5):
    axs[0, j].imshow(x_test_rand[j+13], cmap='binary')
    axs[1, j].imshow(x_predict[j+13], cmap='binary')
    axs[0, j].axis('off')
    axs[1, j].axis('off')